### QSVM4EO

This notebook will present the an example of using quantum-enhanced support vector machines (QSVM) for classification tasks on multi-spectral Earth Observation (EO) data. The main topics covered are introduction to classical SVM, where quantum computation could be used in the SVM calculations, the intricacies of preparing and encoding the classical data into useful quantum states and training QSVMs on gate-based quantum software simulators. Finally, some results on comparing classical and quantum-enhanced SVM models will be presented.

QSVM code is adapted from Qiskit QSVM example: https://qiskit-community.github.io/qiskit-machine-learning/tutorials/03_quantum_kernel.html

In [1]:
import qiskit

from qiskit_machine_learning.utils import algorithm_globals

algorithm_globals.random_seed = 12345

In [2]:
# from qiskit_machine_learning.datasets import ad_hoc_data

# adhoc_dimension = 2
# train_features, train_labels, test_features, test_labels, adhoc_total = ad_hoc_data(
#     training_size=20,
#     test_size=5,
#     n=adhoc_dimension,
#     gap=0.3,
#     plot_data=False,
#     one_hot=False,
#     include_sample_total=True,
# )

In [3]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import numpy as np

def read_csv(file_dir, selected_columns):
    df = pd.read_csv(file_dir)
    df_selected = df[selected_columns]
    X = df_selected.values
    Y = df['Label'].values
    return X,Y

def generate_splits(
    X, Y, test_size, random_state=18051967, trim = None):
    if trim:
        X = X[:trim]
        Y = Y[:trim]
    X_train, X_val, Y_train, Y_val = train_test_split(
        X, Y, test_size=test_size, random_state=random_state
    )
    return X_train, X_val, Y_train, Y_val




In [4]:
train_data = 'data/chips_4369/band_indices_4369_32.csv'
test_data = 'data/chips_4883/band_indices_4883_32.csv'

four_features_data = ['B02', 'B03', 'B04', 'B08']
adhoc_dimension = 4
eight_features_data = ['B02', 'B03', 'B04', 'B08', 'NDVI', 'EVI', 'SAVI', 'NDWI']



In [5]:
train_features, train_labels = read_csv(train_data, four_features_data)

test_features, test_labels = read_csv(test_data, four_features_data)

In [6]:
from sklearn.svm import SVC

svc_classifier = SVC(kernel='sigmoid')
svc_classifier.fit(train_features, train_labels)

train_score = svc_classifier.score(train_features, train_labels)
test_score = svc_classifier.score(test_features, test_labels)


print(f"Precomputed classical kernel classification test score: {test_score}")



Precomputed classical kernel classification test score: 0.3359375


In [7]:
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel, FidelityStatevectorKernel

#adhoc_feature_map = ZZFeatureMap(feature_dimension=adhoc_dimension, reps=2, entanglement="linear")
adhoc_feature_map = ZFeatureMap(feature_dimension=adhoc_dimension, reps=2)

#sampler = Sampler()

#fidelity = ComputeUncompute(sampler=sampler)

# adhoc_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=adhoc_feature_map)
adhoc_kernel = FidelityStatevectorKernel(feature_map=adhoc_feature_map)

adhoc_kernel.feature_map.decompose().draw()


┌───┐┌─────────────┐┌───┐┌─────────────┐
q_0: ┤ H ├┤ P(2.0*x[0]) ├┤ H ├┤ P(2.0*x[0]) ├
     ├───┤├─────────────┤├───┤├─────────────┤
q_1: ┤ H ├┤ P(2.0*x[1]) ├┤ H ├┤ P(2.0*x[1]) ├
     ├───┤├─────────────┤├───┤├─────────────┤
q_2: ┤ H ├┤ P(2.0*x[2]) ├┤ H ├┤ P(2.0*x[2]) ├
     ├───┤├─────────────┤├───┤├─────────────┤
q_3: ┤ H ├┤ P(2.0*x[3]) ├┤ H ├┤ P(2.0*x[3]) ├
     └───┘└─────────────┘└───┘└─────────────┘

In [8]:
adhoc_matrix_train = adhoc_kernel.evaluate(x_vec=train_features)
adhoc_matrix_test = adhoc_kernel.evaluate(x_vec=test_features, y_vec=train_features)

In [9]:
adhoc_svc = SVC(kernel="precomputed")

adhoc_svc.fit(adhoc_matrix_train, train_labels)

adhoc_score_precomputed_kernel = adhoc_svc.score(adhoc_matrix_test, test_labels)

print(f"Precomputed quantum kernel classification test score: {adhoc_score_precomputed_kernel}")

Precomputed quantum kernel classification test score: 0.359375
